In [343]:
!pip install psycopg2 requests pandas numpy scikit-learn matplotlib joblib 

In [314]:
# Check if Docker Desktop is installed
import subprocess
docker_desktop_path = "C:\\Program Files\\Docker\\Docker\\resources\\bin\\docker.exe"
result = subprocess.run([docker_desktop_path, "--version"], capture_output=True, text=True)
print(result.stdout)

Docker version 27.1.1, build 6312585



In [315]:
#Setup DB docker container
result = subprocess.run([docker_desktop_path, "pull", "postgres:latest"], capture_output=True, text=True)
print(result.stdout)

print("setting up db")
result2 = subprocess.run([docker_desktop_path, "run", "-d", "--name", "my_postgres", 
                          "-e", "POSTGRES_USER=myuser", "-e", "POSTGRES_PASSWORD=mypassword", 
                          "-e", "POSTGRES_DB=mydatabase", "-v", "pgdata:/var/lib/postgresql/data", 
                          "-p", "5432:5432", "postgres:latest"], capture_output=True, text=True)
print(result2.stdout)

print("check docker running containers")
result3 = subprocess.run([docker_desktop_path, "ps"], capture_output=True, text=True)
print(result3.stdout)


latest: Pulling from library/postgres
Digest: sha256:026d0ab72b34310b68160ab9299aa1add5544e4dc3243456b94f83cb1c119c2c
Status: Image is up to date for postgres:latest
docker.io/library/postgres:latest

setting up db

check docker running containers
CONTAINER ID   IMAGE             COMMAND                  CREATED        STATUS        PORTS                    NAMES
c3137f1d9797   postgres:latest   "docker-entrypoint.s…"   28 hours ago   Up 19 hours   0.0.0.0:5432->5432/tcp   my_postgres



Download the JSON file from the website
Comment after downloaded to prevent re-download attempt after token expired

In [316]:
'''import requests

url = ["https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=1",
       "https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=2",
       "https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=3",
       "https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=4"]

# Headers as specified in the curl command
headers = {
    "AccessKey": "****************",
    "Token": "*************",
    "User-Agent": "curl/8.7.1",
    "Accept": "*/*",
}

# Make the GET request
for i, u in enumerate(url):
    print(f"Attemping {u}")
    response = requests.get(u, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Save the response to a file
        with open(f"transaction_downloads{i+1}.json", "w") as f:
            f.write(response.text)
        print(f"Data successfully downloaded and saved for batch {i+1}.")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}, Error: {response.text}")'''

'import requests\n\nurl = ["https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=1",\n       "https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=2",\n       "https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=3",\n       "https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=4"]\n\n# Headers as specified in the curl command\nheaders = {\n    "AccessKey": "a716aeab-c83e-4897-94bc-2bb469bc00f5",\n    "Token": "y7-gak4Aea3599RE8HQvb5qcm4464@93rpb7H+xy6fn0b-4-XdZ997Z55e98ta2wjb@yRcaSTd8n8pT3bkq4wf---34M07cc059b",\n    "User-Agent": "curl/8.7.1",\n    "Accept": "*/*",\n}\n\n# Make the GET request\nfor i, u in enumerate(url):\n    print(f"Attemping {u}")\n    response = requests.get(u, headers=headers)\n\n    # Check if the request was successful\n    if response.status_code == 200:\n        # Save the response to a file\n        with open(f"transaction_downloads

Create a connection session and load data. we only use batch 3 and 4 as only these 2 batches contain transaction of ECs

In [26]:
# Imports
import json
import psycopg2

# Load JSON data from the file
with open('transaction_downloads3.json', 'r') as f:
    data3 = json.load(f)
with open('transaction_downloads4.json', 'r') as f:
    data4 = json.load(f)
data = {}
data['Result'] = data3['Result'] + data4['Result']

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="mydatabase",
    user="myuser",
    password="mypassword"
)
cur = conn.cursor()

In [27]:
table_ver = 3
sql_create_table_query = f"""CREATE TABLE transactions_v{table_ver} (
    transaction_id SERIAL PRIMARY KEY,
    project VARCHAR(100),
    market_segment VARCHAR(10),
    contract_date CHAR(4),  -- Store MMYY format
    commence_date CHAR(4),  -- Store MMYY format
    months_since_contract_date INTEGER,  
    months_since_commence_date INTEGER,  
    area INTEGER,
    price NUMERIC(10, 2),
    property_type VARCHAR(50),
    type_of_area VARCHAR(50),
    tenure VARCHAR(100),
    floor_range VARCHAR(10),
    type_of_sale INTEGER,
    district VARCHAR(10),
    no_of_units INTEGER,
    street VARCHAR(100),
    x_coordinate NUMERIC(12, 8),
    y_coordinate NUMERIC(12, 8)
);
"""
sql_check_table_name_query ="""SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';
"""
sql_check_field_name_query=f"""SELECT column_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name = 'transactions_v{table_ver}';
"""

Generate table

In [28]:
# Check if table exists
check_table_query = f"""
SELECT EXISTS (
    SELECT FROM information_schema.tables 
    WHERE table_schema = 'public'
    AND table_name = 'transactions_v{table_ver}'
);
"""
cur.execute(check_table_query)
table_exists = cur.fetchone()[0]

if not table_exists:
    # Table doesn't exist, so create it
    cur.execute(sql_create_table_query)
    conn.commit()
    print(f"Table 'transactions_v{table_ver}' created successfully.")
else:
    print(f"Table 'transactions_v{table_ver}' already exists.")

# ... existing code ...


Table 'transactions_v3' already exists.


In [29]:
sql_create_table_query_10yrs = f"""CREATE TABLE transactions_v{table_ver + 1} (
    transaction_id SERIAL PRIMARY KEY,
    project VARCHAR(100),
    market_segment VARCHAR(10),
    contract_date CHAR(4),  -- Store MMYY format
    commence_date CHAR(4),  -- Store MMYY format
    months_since_contract_date INTEGER,  
    months_since_commence_date INTEGER,  
    area INTEGER,
    price NUMERIC(10, 2),
    property_type VARCHAR(50),
    type_of_area VARCHAR(50),
    tenure VARCHAR(100),
    floor_range VARCHAR(10),
    type_of_sale INTEGER,
    district VARCHAR(10),
    no_of_units INTEGER,
    street VARCHAR(100),
    x_coordinate NUMERIC(12, 8),
    y_coordinate NUMERIC(12, 8)
);
"""

sql_check_field_name_query_10yrs=f"""SELECT column_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name = 'transactions_v{table_ver + 1}';
"""


Generate table for 10 years mark data rows

In [30]:
# Check if table exists
check_table_query = f"""
SELECT EXISTS (
    SELECT FROM information_schema.tables 
    WHERE table_schema = 'public'
    AND table_name = 'transactions_v{table_ver + 1}'
);
"""
cur.execute(check_table_query)
table_exists = cur.fetchone()[0]

if not table_exists:
    # Table doesn't exist, so create it
    cur.execute(sql_create_table_query_10yrs)
    conn.commit()
    print(f"Table 'transactions_v{table_ver + 1}' created successfully.")
else:
    print(f"Table 'transactions_v{table_ver + 1}' already exists.")
    

Table 'transactions_v4' already exists.


Check out the tables and table fields

In [31]:
cur.execute(sql_check_table_name_query)

table_names = cur.fetchall()
print("Tables in the database:")
for table in table_names:
    print(table[0])
    
# Execute the query for the 'transactions' table
cur.execute(sql_check_field_name_query)

# Fetch all results
field_info = cur.fetchall()

# Execute the query for the 'transactions' table
cur.execute(sql_check_field_name_query_10yrs)

# Fetch all results
field_info_10yrs = cur.fetchall()


# Print the results
print(f"\nFields in the 'transactions v{table_ver}' table:")
for field in field_info:
    print(f"Column: {field[0]}, Type: {field[1]}, Max Length: {field[2]}")
    
print(f"\nFields in the 'transactions v{table_ver + 1}' table:")
for field in field_info_10yrs:
    print(f"Column: {field[0]}, Type: {field[1]}, Max Length: {field[2]}")
    

Tables in the database:
transactions
json_staging
transactions_v3
transactions_v4

Fields in the 'transactions v3' table:
Column: transaction_id, Type: integer, Max Length: None
Column: months_since_contract_date, Type: integer, Max Length: None
Column: months_since_commence_date, Type: integer, Max Length: None
Column: area, Type: integer, Max Length: None
Column: price, Type: integer, Max Length: None
Column: type_of_sale, Type: integer, Max Length: None
Column: no_of_units, Type: integer, Max Length: None
Column: x_coordinate, Type: numeric, Max Length: None
Column: y_coordinate, Type: numeric, Max Length: None
Column: property_type, Type: character varying, Max Length: 50
Column: type_of_area, Type: character varying, Max Length: 50
Column: project, Type: character varying, Max Length: 100
Column: market_segment, Type: character varying, Max Length: 10
Column: contract_date, Type: character, Max Length: 4
Column: commence_date, Type: character, Max Length: 4
Column: tenure, Type: c

Quick exploration of the data. Look through first 10 items


In [32]:
for i, item in enumerate(data['Result']):
    print(item)
    if i > 10:
        break
print(f"Fields: {item.keys()} total items {len(item.keys())}")
print(f"Transaction fields {item['transaction'][0].keys()} total items {len(item['transaction'][0].keys())}")


{'street': 'ELLIOT ROAD', 'x': '38874.48712', 'project': 'ELLIOT AT THE EAST COAST', 'y': '32613.09564', 'transaction': [{'area': '92', 'floorRange': '01-05', 'noOfUnits': '1', 'contractDate': '0919', 'typeOfSale': '3', 'price': '1650000', 'propertyType': 'Condominium', 'district': '15', 'typeOfArea': 'Strata', 'tenure': 'Freehold'}, {'area': '126', 'floorRange': '01-05', 'noOfUnits': '1', 'contractDate': '0820', 'typeOfSale': '3', 'price': '1875000', 'propertyType': 'Condominium', 'district': '15', 'typeOfArea': 'Strata', 'tenure': 'Freehold'}, {'area': '236', 'floorRange': '01-05', 'noOfUnits': '1', 'contractDate': '0820', 'typeOfSale': '3', 'price': '2700000', 'propertyType': 'Condominium', 'district': '15', 'typeOfArea': 'Strata', 'tenure': 'Freehold'}, {'area': '47', 'floorRange': '01-05', 'noOfUnits': '1', 'contractDate': '1219', 'typeOfSale': '3', 'price': '810000', 'propertyType': 'Condominium', 'district': '15', 'typeOfArea': 'Strata', 'tenure': 'Freehold'}, {'area': '91', 'fl

function used to convert mmyy to months_from then to now

In [33]:
def convert_mmyy_then2now(mmyy:str)-> int:
    mm, yy = int(mmyy[0:2]), int(mmyy[2:])
    then2now_monnths = (24 - yy) * 12 + (9 - mm)
    return then2now_monnths

    

Begin populating in each transaction as one row in the transaction table  
This cell populate a table of EC transaction which contract date occurs 5 years - 7.5 years from commencement date  
Data rows from this table are used to train a ML model to predict resale price of ECs at 5 years mark


In [34]:
transaction_id = 0
conn.rollback()
# Modify the table schema if necessary
alter_table_query = f"""
ALTER TABLE transactions_v{table_ver}
ALTER COLUMN x_coordinate TYPE NUMERIC(11, 4),
ALTER COLUMN y_coordinate TYPE NUMERIC(11, 4),
ALTER COLUMN price TYPE INTEGER;
"""
cur.execute(alter_table_query)
conn.commit()

# Delete all old rows from the table
delete_query = f"DELETE FROM transactions_v{table_ver};"
cur.execute(delete_query)
conn.commit()
print(f"All rows deleted from transactions_v{table_ver}")

t_item_required_keys = ['contractDate', 'tenure', 'area', 'price', 'typeOfSale', 'noOfUnits', 'propertyType', 'typeOfArea', 'district', 'floorRange']
item_required_keys = ['street', 'project', 'marketSegment']


for i, item in enumerate(data['Result']):
    if not all(key in item.keys() for key in item_required_keys):
        print(f"skipping item not complete  {item.keys()}")
        continue
    for x, transaction_item in enumerate(item['transaction']):
        if not all(key in transaction_item.keys() for key in t_item_required_keys):
            print(f"skipping t item not complete {transaction_item.keys()}")
            continue
        
        # Filter away non lease base condo
        if transaction_item['tenure'] == "Freehold":
            continue
        # some rows has no commencing from year, filter them out
        if "commencing from" not in transaction_item['tenure']:
            continue
        # lease before singapore founding are irrelevant
        if int(transaction_item['tenure'][-4:]) < 1965:
            continue

        # We only want to look at ECs to predict resale price at 5th year mark since commencement
        if transaction_item['propertyType'] != "Executive Condominium":
            continue
        # Filter away contract date not within specified point 
        # where contract date is not 5 years from lease commencement
        months_since_contract_date = convert_mmyy_then2now(transaction_item['contractDate'])
        months_since_commence_date = convert_mmyy_then2now("01" + transaction_item['tenure'][-2:])
        if months_since_commence_date - months_since_contract_date < 60:
            continue
        if months_since_commence_date - months_since_contract_date > 90:
            continue
            
        
        # Filter away contract dates older than 5 years ago
        if months_since_contract_date > 60:
            continue
        
        
        area = int(round(float(transaction_item['area'])))
        price = int(transaction_item['price'])
        type_of_sale = transaction_item['typeOfSale']
        no_of_units = transaction_item['noOfUnits']
        x_coordinate = item['x'] if 'x' in item.keys() else 0
        y_coordinate = item['y'] if 'y' in item.keys() else 0
        property_type = transaction_item['propertyType']
        type_of_area = transaction_item['typeOfArea']
        project = item['project']
        market_segment = item['marketSegment']
        contract_date = transaction_item['contractDate']
        commence_date = "06" + transaction_item['tenure'][-2:]
        tenure = transaction_item['tenure']
        floor_range = transaction_item['floorRange']
        street = item['street']
        district = transaction_item['district']
        transaction_tuple = (transaction_id, months_since_contract_date, months_since_commence_date, 
                             area, price, type_of_sale, no_of_units, x_coordinate, y_coordinate, property_type, 
                             type_of_area, project, market_segment, contract_date, commence_date, tenure, 
                             floor_range, street, district)
        # Insert data into the table
        cur.execute(
            f"""
            INSERT INTO transactions_v{table_ver} (transaction_id, months_since_contract_date, months_since_commence_date, 
            area, price, type_of_sale, no_of_units, x_coordinate, y_coordinate, property_type, 
            type_of_area, project, market_segment, contract_date, commence_date, tenure, 
            floor_range, street, district) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            transaction_tuple
        )
        transaction_id += 1
        print(f"finished {transaction_id}")



print(f"FINAL TRANSACTION _ID {transaction_id}")
print(f"Fields: {item.keys()} total items {len(item.keys())}")
print(f"Transaction fields {item['transaction'][0].keys()} total items {len(item['transaction'][0].keys())}")
conn.commit()


All rows deleted from transactions_v3
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
finished 13
finished 14
finished 15
finished 16
finished 17
finished 18
finished 19
finished 20
finished 21
finished 22
finished 23
finished 24
finished 25
finished 26
finished 27
finished 28
finished 29
finished 30
finished 31
finished 32
finished 33
finished 34
finished 35
finished 36
finished 37
finished 38
finished 39
finished 40
finished 41
finished 42
finished 43
finished 44
finished 45
finished 46
finished 47
finished 48
finished 49
finished 50
finished 51
finished 52
finished 53
finished 54
finished 55
finished 56
finished 57
finished 58
finished 59
finished 60
finished 61
finished 62
finished 63
finished 64
finished 65
finished 66
finished 67
finished 68
finished 69
finished 70
finished 71
finished 72
finished 73
finished 74
finished 75
finished 76
finished 77
finished 78
finished 79
finished 80
finished 81

Begin populating in each transaction as one row in the transaction table  
This cell populate a table of EC transaction which contract date occurs 10 years - 12.5 years from commencement date  
Data rows from this table are used to train a ML model to predict resale price of ECs at 10 years mark

In [35]:
transaction_id = 0
# Modify the table schema if necessary
alter_table_query = f"""
ALTER TABLE transactions_v{table_ver + 1}
ALTER COLUMN x_coordinate TYPE NUMERIC(11, 4),
ALTER COLUMN y_coordinate TYPE NUMERIC(11, 4),
ALTER COLUMN price TYPE INTEGER;
"""
conn.rollback()
cur.execute(alter_table_query)
conn.commit()

# Delete all old rows from the table
delete_query = f"DELETE FROM transactions_v{table_ver + 1};"
cur.execute(delete_query)
conn.commit()
print(f"All rows deleted from transactions_v{table_ver + 1}")

t_item_required_keys = ['contractDate', 'tenure', 'area', 'price', 'typeOfSale', 'noOfUnits', 'propertyType', 'typeOfArea', 'district', 'floorRange']
item_required_keys = ['street', 'project', 'marketSegment']


for i, item in enumerate(data['Result']):
    if not all(key in item.keys() for key in item_required_keys):
        print(f"skipping item not complete  {item.keys()}")
        continue
    for x, transaction_item in enumerate(item['transaction']):
        if not all(key in transaction_item.keys() for key in t_item_required_keys):
            print(f"skipping t item not complete {transaction_item.keys()}")
            continue
        
        # Filter away non lease base condo
        if transaction_item['tenure'] == "Freehold":
            continue
        # some rows has no commencing from year, filter them out
        if "commencing from" not in transaction_item['tenure']:
            continue
        # lease before singapore founding are irrelevant
        if int(transaction_item['tenure'][-4:]) < 1965:
            continue

        # We only want to look at ECs to predict resale price at 5th year mark since commencement
        if transaction_item['propertyType'] != "Executive Condominium":
            continue
        # Filter away contract date not within specified point 
        # where contract date is not 5 years from lease commencement
        months_since_contract_date = convert_mmyy_then2now(transaction_item['contractDate'])
        months_since_commence_date = convert_mmyy_then2now("01" + transaction_item['tenure'][-2:])
        if months_since_commence_date - months_since_contract_date < 120:
            continue
        if months_since_commence_date - months_since_contract_date > 150:
            continue
        
        # Filter away contract dates older than 5 years ago
        if months_since_contract_date > 60:
            continue
        
        
        area = int(round(float(transaction_item['area'])))
        price = int(transaction_item['price'])
        type_of_sale = transaction_item['typeOfSale']
        no_of_units = transaction_item['noOfUnits']
        x_coordinate = item['x'] if 'x' in item.keys() else 0
        y_coordinate = item['y'] if 'y' in item.keys() else 0
        property_type = transaction_item['propertyType']
        type_of_area = transaction_item['typeOfArea']
        project = item['project']
        market_segment = item['marketSegment']
        contract_date = transaction_item['contractDate']
        commence_date = "06" + transaction_item['tenure'][-2:]
        tenure = transaction_item['tenure']
        floor_range = transaction_item['floorRange']
        street = item['street']
        district = transaction_item['district']
        transaction_tuple = (transaction_id, months_since_contract_date, months_since_commence_date, 
                             area, price, type_of_sale, no_of_units, x_coordinate, y_coordinate, property_type, 
                             type_of_area, project, market_segment, contract_date, commence_date, tenure, 
                             floor_range, street, district)
        # Insert data into the table
        cur.execute(
            f"""
            INSERT INTO transactions_v{table_ver + 1} (transaction_id, months_since_contract_date, months_since_commence_date, 
            area, price, type_of_sale, no_of_units, x_coordinate, y_coordinate, property_type, 
            type_of_area, project, market_segment, contract_date, commence_date, tenure, 
            floor_range, street, district) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            transaction_tuple
        )
        transaction_id += 1
        print(f"finished {transaction_id}")


print(f"FINAL TRANSACTION _ID {transaction_id}")
print(f"Fields: {item.keys()} total items {len(item.keys())}")
print(f"Transaction fields {item['transaction'][0].keys()} total items {len(item['transaction'][0].keys())}")
conn.commit()


All rows deleted from transactions_v4
finished 1
finished 2
finished 3
finished 4
finished 5
finished 6
finished 7
finished 8
finished 9
finished 10
finished 11
finished 12
finished 13
finished 14
finished 15
finished 16
finished 17
finished 18
finished 19
finished 20
finished 21
finished 22
finished 23
finished 24
finished 25
finished 26
finished 27
finished 28
finished 29
finished 30
finished 31
finished 32
finished 33
finished 34
finished 35
finished 36
finished 37
finished 38
finished 39
finished 40
finished 41
finished 42
finished 43
finished 44
finished 45
finished 46
finished 47
finished 48
finished 49
finished 50
finished 51
finished 52
finished 53
finished 54
finished 55
finished 56
finished 57
finished 58
finished 59
finished 60
finished 61
finished 62
finished 63
finished 64
finished 65
finished 66
finished 67
finished 68
finished 69
finished 70
finished 71
finished 72
finished 73
finished 74
finished 75
finished 76
finished 77
finished 78
finished 79
finished 80
finished 81

In [36]:
# Count rows in the table
count_query = f"SELECT COUNT(*) FROM transactions_v{table_ver};"
cur.execute(count_query)
row_count = cur.fetchone()[0]

print(f"Number of rows in transactions_v{table_ver}: {row_count}")

# Count rows in the table
count_query = f"SELECT COUNT(*) FROM transactions_v{table_ver +1};"
cur.execute(count_query)
row_count = cur.fetchone()[0]

print(f"Number of rows in transactions_v{table_ver + 1}: {row_count}")

Number of rows in transactions_v3: 106
Number of rows in transactions_v4: 2654


Generate random 50 sample rows to inspect (Do run this multiple times)

In [37]:
# Generate 50 random sample rows
sample_query = f"""
SELECT *
FROM transactions_v{table_ver}
ORDER BY RANDOM()
LIMIT 50;
"""

cur.execute(sample_query)
sample_rows = cur.fetchall()

# Print the sample rows
print(f"Sample of 50 rows from transactions_v{table_ver}:")
for row in sample_rows:
    print(row)

# Optionally, you can also print the column names
column_names = [desc[0] for desc in cur.description]
print("\nColumn names:")
print(column_names)

Sample of 50 rows from transactions_v3:
(53, 'SOL ACRES', 'OCR', '0721', '0614', 38, 128, 46, 560000, 'Executive Condominium', 'Strata', '99 yrs lease commencing from 2014', '21-25', 3, '23', 1, 'CHOA CHU KANG GROVE', Decimal('18803.7060'), Decimal('39913.3230'))
(56, 'SOL ACRES', 'OCR', '0321', '0614', 42, 128, 74, 718000, 'Executive Condominium', 'Strata', '99 yrs lease commencing from 2014', '01-05', 3, '23', 1, 'CHOA CHU KANG GROVE', Decimal('18803.7060'), Decimal('39913.3230'))
(28, 'THE BROWNSTONE', 'OCR', '0121', '0614', 44, 128, 99, 1160000, 'Executive Condominium', 'Strata', '99 yrs lease commencing from 2014', '01-05', 3, '27', 1, 'CANBERRA DRIVE', Decimal('27325.6909'), Decimal('47435.8188'))
(59, 'THE VISIONAIRE', 'OCR', '0121', '0615', 44, 116, 91, 980000, 'Executive Condominium', 'Strata', '99 yrs lease commencing from 2015', '06-10', 3, '27', 1, 'CANBERRA DRIVE', Decimal('27164.7477'), Decimal('47484.9129'))
(64, 'THE VISIONAIRE', 'OCR', '0321', '0615', 42, 116, 79, 8180

Final check to see if the generated rows are good

In [38]:
# Generate 5 detailed random sample rows
sample_query = f"""
SELECT *
FROM transactions_v{table_ver}
ORDER BY RANDOM()
LIMIT 5;
"""

cur.execute(sample_query)
sample_rows = cur.fetchall()

# Get column names
column_names = [desc[0] for desc in cur.description]

# Print the sample rows with field names
print(f"Sample of up to 5 rows from transactions_v{table_ver}:")
for row in sample_rows:
    print("\n--- New Row ---")
    for field_name, value in zip(column_names, row):
        print(f"{field_name}: {value}")

# Print summary
print(f"\nTotal sample rows: {len(sample_rows)}")
print(f"Fields: {', '.join(column_names)}")


sample_query = f"""
SELECT *
FROM transactions_v{table_ver + 1}
ORDER BY RANDOM()
LIMIT 5;
"""

cur.execute(sample_query)
sample_rows = cur.fetchall()

# Get column names
column_names = [desc[0] for desc in cur.description]

# Print the sample rows with field names
print(f"\n\nSample of up to 5 rows from transactions_v{table_ver + 1}:")
for row in sample_rows:
    print("\n--- New Row ---")
    for field_name, value in zip(column_names, row):
        print(f"{field_name}: {value}")

# Print summary
print(f"\nTotal sample rows: {len(sample_rows)}")
print(f"Fields: {', '.join(column_names)}")

Sample of up to 5 rows from transactions_v3:

--- New Row ---
transaction_id: 13
project: RIVERCOVE RESIDENCES
market_segment: OCR
contract_date: 0221
commence_date: 0616
months_since_contract_date: 43
months_since_commence_date: 104
area: 108
price: 1329000
property_type: Executive Condominium
type_of_area: Strata
tenure: 99 yrs lease commencing from 2016
floor_range: 16-20
type_of_sale: 1
district: 19
no_of_units: 1
street: ANCHORVALE LANE
x_coordinate: 33600.4728
y_coordinate: 41713.5921

--- New Row ---
transaction_id: 94
project: NORTHWAVE
market_segment: OCR
contract_date: 0722
commence_date: 0615
months_since_contract_date: 26
months_since_commence_date: 116
area: 103
price: 1150000
property_type: Executive Condominium
type_of_area: Strata
tenure: 99 yrs lease commencing from 2015
floor_range: 01-05
type_of_sale: 3
district: 25
no_of_units: 1
street: WOODLANDS VIEW
x_coordinate: 25422.0177
y_coordinate: 47229.6238

--- New Row ---
transaction_id: 99
project: SIGNATURE AT YISHUN
